In [1]:
import numpy as np
np.random.seed(0)

# Log plot scale
def xlog_scale(log_x_max, scale, log_base= 10): 
    '''Logaritmic scale up to log_alpha_max'''
    bd_block = np.arange(0, log_base**2, log_base) + log_base
    bd_block = bd_block[0:-1]
    xlog = np.tile(bd_block, log_x_max)
    xlog[(log_base-1) : 2*(log_base-1)] = log_base*xlog[(log_base-1) : 2*(log_base-1)]
    for j in range(1, log_x_max - 1):
        xlog[(j+1)*(log_base-1) : (j+2)*(log_base-1)] = log_base*xlog[  j*(log_base-1) :  (j+1)*(log_base-1)  ]
    xlog = np.insert(xlog, 0,  np.arange(1,log_base), axis=0)
    xlog = np.insert(xlog, len(xlog),log_base**(log_x_max+1), axis=0)
    jlog = (xlog*scale).astype(int)
    return jlog

### Build Cython code

If not done before, build the cython code on the command line.

In [11]:
! python setup.py build_ext --inplace

running build_ext


### Import simulation class

In [2]:
from trainSGD import trainSGD

### Parameters

In [3]:
# Data dimension
d = 100
# Number of data points
n = 40
# Choose p 
p = 10
# Number of S random instances
nS = 10
# Signal-to-noise ratio
snr = 1./5.
# Max number of (S)GD iterations
n_it_max= int(1e6)  
##### Number of SGD runs 
n_SGD = 1
##### Store only the terminal point in the dynamics
only_end = False
##### Learning rate
lr = 1./d
##### Data matrix
X = np.random.randn(n,d)
##### Beta star d-dimension (ground true)
beta_star_ = np.random.randn(d)
beta_star = np.copy(beta_star_) / np.linalg.norm(beta_star_)
###### Initializing the estimator
beta0_ = np.random.randn(d)
beta0  = np.copy(beta0_)/np.linalg.norm(beta0_)
##### Targets
y = X @ beta_star + snr*np.random.randn(n)
#### Plot
log_x_max = (np.log10(n_it_max)-1).astype(int)  
plot_list = xlog_scale(log_x_max, scale=1., log_base= 10)
if only_end:
    plot_list = plot_list[-1:]
plot_list = np.insert(plot_list, 0, 0)

In [4]:
print(plot_list)
print(plot_list.shape)

[      0       1       2       3       4       5       6       7       8
       9      10      20      30      40      50      60      70      80
      90     100     200     300     400     500     600     700     800
     900    1000    2000    3000    4000    5000    6000    7000    8000
    9000   10000   20000   30000   40000   50000   60000   70000   80000
   90000  100000  200000  300000  400000  500000  600000  700000  800000
  900000 1000000]
(56,)


### Training

In [5]:
betaSGD = trainSGD(p=p, nS=nS, lr=lr, X=X, y=y, beta0= beta0, plot_list=plot_list, n=n, d=d, n_it_max=n_it_max, n_SGD= n_SGD)

Beginning SGD training ----- p= 10 --- nS= 10
Time elapsed SGD: 26.150902
End ----- p= 10 --- nS= 10
 


In [6]:
betaSGD

array([[ 7.15469408e-02,  7.68967350e-02, -9.04543553e-02,
        -7.75515762e-02, -5.97368481e-03,  1.29601259e-01,
         2.85261241e-03,  1.71508410e-01, -1.47512991e-02,
         9.45496437e-02],
       [ 7.17769275e-02,  7.79102918e-02, -9.32549416e-02,
        -7.93281378e-02, -6.55724601e-03,  1.28897345e-01,
         2.64853095e-03,  1.75001546e-01, -1.54641720e-02,
         9.20602104e-02],
       [ 7.34498782e-02,  7.63567866e-02, -9.88516078e-02,
        -7.27579290e-02, -4.28875707e-03,  1.33401616e-01,
        -1.08913606e-03,  1.81540199e-01, -1.95922661e-02,
         9.73373690e-02],
       [ 6.78776010e-02,  7.57902022e-02, -1.06485312e-01,
        -7.31282221e-02, -8.81663812e-04,  1.28866075e-01,
        -1.39629445e-04,  1.78028751e-01, -1.41229270e-02,
         9.51600329e-02],
       [ 6.46218349e-02,  7.84793640e-02, -1.06901405e-01,
        -7.09434053e-02, -8.41117161e-03,  1.32619883e-01,
        -3.21583933e-03,  1.80985562e-01, -1.00402169e-02,
         9.

In [7]:
print(betaSGD.shape)

(57, 10)


In [8]:
with open('results/'+'betaSGD_howto.npy', 'wb') as f:
    np.save(f, betaSGD)   

### In the paper

To get the training weights used in the paper, use the script `scrSGD.py` after building the Cython code. One might want to appropriately set `intI` and `intF` to get a particular interval in `alpha`. 

In [ ]:
! python scrSGD.py